<a href="https://colab.research.google.com/github/Akshess/Change-Analysis-of-Water-Bodies-CNN-OBIA/blob/main/Water_Transformation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import imageio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Function to calculate the rate of difference in water regions
def calculate_water_difference_rate(water_mask):
    total_pixels = water_mask.size
    different_pixels = np.count_nonzero(water_mask == 255)
    difference_rate = (different_pixels / total_pixels) * 100
    return difference_rate

# Function to perform object-based image analysis (OBIA) using NDWI
def obia(image1, image2, ndwi_threshold=0.02, save_path=None):
    # Ensure both images have the same dimensions
    min_height = min(image1.shape[0], image2.shape[0])
    min_width = min(image1.shape[1], image2.shape[1])
    image1 = cv2.resize(image1, (min_width, min_height))
    image2 = cv2.resize(image2, (min_width, min_height))

    # Check if the images have 3 channels (RGB)
    if image1.shape[2] == 3 and image2.shape[2] == 3:
        # Convert images to HSV color space
        hsv1 = cv2.cvtColor(image1, cv2.COLOR_RGB2HSV)
        hsv2 = cv2.cvtColor(image2, cv2.COLOR_RGB2HSV)

        # Define range of blue color in HSV
        lower_blue = np.array([90, 50, 50])
        upper_blue = np.array([130, 255, 255])

        # Create masks for blue regions in HSV
        blue_mask1 = cv2.inRange(hsv1, lower_blue, upper_blue)
        blue_mask2 = cv2.inRange(hsv2, lower_blue, upper_blue)

        # Combine NDWI and HSV blue masks
        water_mask1 = cv2.bitwise_or(blue_mask1, (ndwi(image1) >= ndwi_threshold).astype(np.uint8) * 255)
        water_mask2 = cv2.bitwise_or(blue_mask2, (ndwi(image2) >= ndwi_threshold).astype(np.uint8) * 255)

        # Perform morphological operations for refinement
        kernel = np.ones((10, 10), np.uint8)  # Larger kernel for closing
        water_mask1 = cv2.morphologyEx(water_mask1, cv2.MORPH_CLOSE, kernel)
        water_mask2 = cv2.morphologyEx(water_mask2, cv2.MORPH_CLOSE, kernel)

        # Highlight water changes in blue
        water_difference = cv2.absdiff(water_mask1, water_mask2)
        water_highlighted = np.zeros_like(image2)
        water_highlighted[water_difference == 255, 2] = 255  # Set the blue channel based on the water difference

        # Save the water-highlighted images
        if save_path is not None:
            filename1 = os.path.join(save_path, f'water_highlighted_1_{os.path.basename(image1_path)}')
            filename2 = os.path.join(save_path, f'water_highlighted_2_{os.path.basename(image2_path)}')
            filename_diff = os.path.join(save_path, f'water_difference_{os.path.basename(image1_path)}_{os.path.basename(image2_path)}')

            cv2.imwrite(filename1, cv2.cvtColor(water_mask1, cv2.COLOR_RGB2BGR))
            cv2.imwrite(filename2, cv2.cvtColor(water_mask2, cv2.COLOR_RGB2BGR))
            cv2.imwrite(filename_diff, cv2.cvtColor(water_highlighted, cv2.COLOR_RGB2BGR))

        # Display the images with water-highlighted regions
        plt.figure(figsize=(15, 6))
        plt.subplot(131), plt.imshow(water_mask1), plt.title('Water Mask Image 1')
        plt.subplot(132), plt.imshow(water_mask2), plt.title('Water Mask Image 2')
        plt.subplot(133), plt.imshow(water_highlighted), plt.title('Water Difference Image')
        plt.show()

        # Calculate and print the rate of difference for water
        water_difference_rate = calculate_water_difference_rate(water_difference)
        print(f"Rate of Water Difference: {water_difference_rate:.2f}%")

        return water_mask1, water_mask2, water_difference_rate  # Return the water masks and difference rate

    else:
        print("Images do not have the expected number of channels (RGB). Please check your input images.")

# NDWI function
def ndwi(image):
    green = image[:, :, 1].astype(float)
    blue = image[:, :, 2].astype(float)
    return (green - blue) / (green + blue)

# Function to load and preprocess images
def load_and_preprocess_data(folder_path_water, folder_path_non_water):
    images = []
    labels = []

    # Load water images
    for image_file in os.listdir(folder_path_water):
        image_path = os.path.join(folder_path_water, image_file)
        try:
            img = imageio.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(1)  # Assign label 1 for water images
        except Exception as e:
            print(f"Error loading image: {image_path}. Exception: {e}")

    # Load non-water images
    for image_file in os.listdir(folder_path_non_water):
        image_path = os.path.join(folder_path_non_water, image_file)
        try:
            img = imageio.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(0)  # Assign label 0 for non-water images
        except Exception as e:
            print(f"Error loading image: {image_path}. Exception: {e}")

    return np.array(images), np.array(labels)

# Define CNN model architecture for water classification
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')  # Output layer with 2 classes: water and non-water
    ])
    return model

# Specify the paths to the two folders containing TIFF images
folder_path_water = "/content/drive/MyDrive/Folder1"  # Update with your folder path containing water images
folder_path_non_water = "/content/drive/MyDrive/Folder2"  # Update with your folder path containing non-water images

# Specify the path to save the water-highlighted images
save_path = "/content/drive/MyDrive/HighlightedImages"

# Create the directory if it does not exist
os.makedirs(save_path, exist_ok=True)

# Load and preprocess data for CNN
images_cnn, labels_cnn = load_and_preprocess_data(folder_path_water, folder_path_non_water)

# Check if data is not empty before training
if len(images_cnn) > 0 and len(labels_cnn) > 0:
    # Convert labels to one-hot encoding
    labels_cnn_one_hot = tf.keras.utils.to_categorical(labels_cnn, num_classes=2)

    # Split data into training and testing sets
    X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(images_cnn, labels_cnn_one_hot, test_size=0.2, random_state=42)

    # Define CNN model
    model_cnn = create_cnn_model(input_shape=X_train_cnn[0].shape)

    # Compile the model
    model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with early stopping
    history = model_cnn.fit(X_train_cnn, y_train_cnn, epochs=10, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

    # Evaluate the model
    test_loss_cnn, test_acc_cnn = model_cnn.evaluate(X_test_cnn, y_test_cnn)
    print(f'Test loss: {test_loss_cnn}')
    print(f'Test accuracy: {test_acc_cnn}')

    # Save the trained CNN model for future use
    model_cnn.save("/content/drive/MyDrive/new_water_body_segmentation_model.h5")

    # Iterate over pairs of images from the two folders
    for image_file1 in os.listdir(folder_path_water):
        # Extract the base name of the image without extension
        base_name1 = os.path.splitext(image_file1)[0]

        # Search for a corresponding image in folder_path2
        corresponding_image = next((f for f in os.listdir(folder_path_non_water) if base_name1 in f), None)

        # If a corresponding image is found, proceed with the analysis
        if corresponding_image:
            image_file2 = corresponding_image

            image1_path = os.path.join(folder_path_water, image_file1)
            image2_path = os.path.join(folder_path_non_water, image_file2)

            img1 = imageio.imread(image1_path)
            img2 = imageio.imread(image2_path)

            # Predict water regions using the trained CNN model
            cnn_image1 = cv2.resize(img1, (224, 224))
            cnn_image1 = np.expand_dims(cnn_image1, axis=0)  # Add batch dimension
            cnn_prediction1 = model_cnn.predict(cnn_image1)

            cnn_image2 = cv2.resize(img2, (224, 224))
            cnn_image2 = np.expand_dims(cnn_image2, axis=0)  # Add batch dimension
            cnn_prediction2 = model_cnn.predict(cnn_image2)

            # Visualize water and non-water regions separately
            water_mask1, _, _ = obia(img1, img2, ndwi_threshold=0.02, save_path=save_path)
            water_mask2, _, _ = obia(img2, img1, ndwi_threshold=0.02, save_path=save_path)

            # Plot water regions in image 1
            water_region_img1 = cv2.bitwise_and(img1, img1, mask=water_mask1)
            plt.figure(figsize=(8, 8))
            plt.imshow(cv2.cvtColor(water_region_img1, cv2.COLOR_BGR2RGB))
            plt.title("Water Region in Image 1")
            plt.axis('off')
            plt.show()

            # Plot non-water regions in image 1
            non_water_region_img1 = cv2.bitwise_and(img1, img1, mask=cv2.bitwise_not(water_mask1))
            plt.figure(figsize=(8, 8))
            plt.imshow(cv2.cvtColor(non_water_region_img1, cv2.COLOR_BGR2RGB))
            plt.title("Non-Water Region in Image 1")
            plt.axis('off')
            plt.show()

            # Plot water regions in image 2
            water_region_img2 = cv2.bitwise_and(img2, img2, mask=water_mask2)
            plt.figure(figsize=(8, 8))
            plt.imshow(cv2.cvtColor(water_region_img2, cv2.COLOR_BGR2RGB))
            plt.title("Water Region in Image 2")
            plt.axis('off')
            plt.show()

            # Plot non-water regions in image 2
            non_water_region_img2 = cv2.bitwise_and(img2, img2, mask=cv2.bitwise_not(water_mask2))
            plt.figure(figsize=(8, 8))
            plt.imshow(cv2.cvtColor(non_water_region_img2, cv2.COLOR_BGR2RGB))
            plt.title("Non-Water Region in Image 2")
            plt.axis('off')
            plt.show()

            print(f'CNN Prediction for Image 1: {cnn_prediction1[0]}')
            print(f'CNN Prediction for Image 2: {cnn_prediction2[0]}')

        else:
            print(f"No corresponding image found for {image_file1}")
else:
    print("No images found for CNN training. Please check the specified folder paths.")


<ipython-input-2-130dd3640d3a>:94: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(image_path)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
